In [1]:
import ee
import pandas as pd

# Initialize Earth Engine
ee.Initialize()


In [2]:
# Define a region of interest (ROI)
roi = ee.Geometry.Point([-122.262, 37.8719]).buffer(10000)

# Define the ROIs (already provided)
roi_for_brightness = ee.Geometry.Polygon(
    [[163.05418845709113, -77.62217141675877], 
    [163.04800864752082, -77.61908009873933], 
    [163.0795943408802, -77.61422077772508], 
    [163.08302756841925, -77.61569349746868],
    [163.09813376959113, -77.60950691594756], 
    [163.1407057910755, -77.60567560295046], 
    [163.15924521978644, -77.60228539245952], 
    [163.1736647754505, -77.60140083961204], 
    [163.17435142095832, -77.5996315475443],
    [163.18259116705207, -77.59786200695194],
    [163.19769736822394, -77.60125340809783], 
    [163.21417686041144, -77.60184312380088], 
    [163.2505690723255, -77.59874680831578], 
    [163.25812217291144, -77.60036878277327],
    [163.25400229986457, -77.60302247238047],
    [163.25468894537238, -77.60493867830638], 
    [163.25880881841925, -77.60670722499943], 
    [163.2334029346302, -77.60788611813062], 
    [163.21967002447394, -77.60655985559598],
    [163.20799705084113, -77.60980158403946],
    [163.21280356939582, -77.61156944773495], 
    [163.19357749517707, -77.61716604483516], 
    [163.18121787603644, -77.6155462332516], 
    [163.15237876470832, -77.61687154915111],
    [163.14894553716925, -77.62025783330262],
    [163.12010642584113, -77.62231860342517], 
    [163.10637351568488, -77.62217141675877], 
    [163.0795943408802, -77.61893287416414], 
    [163.06448813970832, -77.61849119009841],
    [163.06723472173957, -77.62099386140082],
    [163.06448813970832, -77.6226129715892],
    [163.05418845709113, -77.62217141675877]]
)


In [3]:
# Load the ImageCollection (Landsat 8 example)
image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA") \
    .filterDate("2023-01-01", "2023-12-31") \
    .filterBounds(roi_for_brightness) \
    .select(['B2', 'B3', 'B4', 'B5', 'B6'])  # Blue, Green, Red, NIR, SWIR


In [4]:
# Function to calculate mean band values
def calculate_mean_band_values(image, point_geometry):
    buffer = point_geometry.buffer(90)  # 3x3 Landsat pixel window
    band_means = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=buffer,
        scale=30,
        maxPixels=1e6
    )
    return band_means

In [5]:
# Function to define endmembers and retain geometries
def define_endmembers(image):
    brightness = image.select(['B2', 'B3', 'B4', 'B5', 'B6']).reduce(ee.Reducer.sum()).rename('brightness')

    # Generate 2000 random points in ROI
    points = ee.FeatureCollection.randomPoints(region=roi_for_brightness, points=2000)

    # Compute brightness at each point
    points_with_brightness = brightness.reduceRegions(
        collection=points,
        reducer=ee.Reducer.mean(),
        scale=30
    )

    # Sort points by brightness
    sorted_points = points_with_brightness.sort('brightness', False)  # Brightest first
    top_bright = sorted_points.toList(1)
    bright_geom = ee.FeatureCollection(top_bright).geometry()

    # Filter out points near the brightest point
    filtered_points = points_with_brightness.filter(ee.Filter.bounds(bright_geom.buffer(100)).Not())
    sorted_filtered = filtered_points.sort('brightness', True)  # Dimmest first
    bottom_dim = sorted_filtered.toList(1)
    dim_geom = ee.FeatureCollection(bottom_dim).geometry()

    # Compute mean band values for the brightest and dimmest points
    bright_means = calculate_mean_band_values(image, bright_geom)
    dim_means = calculate_mean_band_values(image, dim_geom)

    # Create a Feature storing all relevant data
    feature = ee.Feature(None, {
        'date': image.date().format('YYYY-MM-dd'),
        'bright_lat': bright_geom.centroid().coordinates().get(1),
        'bright_lon': bright_geom.centroid().coordinates().get(0),
        'dim_lat': dim_geom.centroid().coordinates().get(1),
        'dim_lon': dim_geom.centroid().coordinates().get(0),
        'bright_B2': bright_means.get('B2'),
        'dim_B2': dim_means.get('B2'),
        'bright_B3': bright_means.get('B3'),
        'dim_B3': dim_means.get('B3')
    })  
    
    return feature

In [8]:
# Map over ImageCollection
features = ee.FeatureCollection(image_collection.map(define_endmembers))
print(features.first().getInfo())


None


In [ ]:
# Export FeatureCollection as CSV
task = ee.batch.Export.table.toDrive(
    collection=features,
    description="Endmembers_Geometry_Export",
    fileFormat="CSV"
)
task.start()

print("Export started. Check your Google Drive.")